## Imports, Config and Seeding

In [124]:
import timm
import torch
import torchvision
from typing import Dict, Union, Callable, OrderedDict
import os, random
import numpy as np

In [125]:
def seed_all(seed: int = 1992) -> None:
    """Seed all random number generators."""
    print(f"Using Seed Number {seed}")

    os.environ["PYTHONHASHSEED"] = str(seed)  # set PYTHONHASHSEED env var at fixed value
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.cuda.manual_seed(seed)  # pytorch (both CPU and CUDA)
    np.random.seed(seed)  # for numpy pseudo-random generator
    # set fixed value for python built-in pseudo-random generator
    random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
    torch.backends.cudnn.enabled = True


def seed_worker(_worker_id) -> None:
    """Seed a worker with the given ID."""
    worker_seed = torch.initial_seed() % 2 ** 32
    np.random.seed(worker_seed)
    random.seed(worker_seed)


seed_all(seed=1992)

Using Seed Number 1992


In [126]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [127]:
resnet18_pretrained_true = timm.create_model(model_name = "resnet34", pretrained=True, num_classes=10).to(DEVICE)

## Toy Models

I created two versions of the same model. The `Sequential` method has a more compact form, but often is more difficult to extract layers.

In [128]:
class ToyModel(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.cl1 = torch.nn.Linear(25, 60)
        self.cl2 = torch.nn.Linear(60, 16)
        self.fc1 = torch.nn.Linear(16, 120)
        self.fc2 = torch.nn.Linear(120, 84)
        self.fc3 = torch.nn.Linear(84, 10)

    def forward(self, x):
        """Forward pass of the model.

        Args:
            x ([type]): [description]

        Returns:
            [type]: [description]
        """
        x = torch.nn.ReLU()(self.cl1(x))
        x = torch.nn.ReLU()(self.cl2(x))
        x = torch.nn.ReLU()(self.fc1(x))
        x = torch.nn.ReLU()(self.fc2(x))
        x = torch.nn.LogSoftmax(dim=1)(self.fc3(x))
        return x


class ToySequentialModel(torch.nn.Module):
    # Create a sequential model pytorch same as ToyModel.
    def __init__(self) -> None:
        super().__init__()

        self.backbone = torch.nn.Sequential(
            OrderedDict(
                [
                    ("cl1", torch.nn.Linear(25, 60)),
                    ("cl_relu1", torch.nn.ReLU()),
                    ("cl2", torch.nn.Linear(60, 16)),
                    ("cl_relu2", torch.nn.ReLU()),
                ]
            )
        )

        self.head = torch.nn.Sequential(
            OrderedDict(
                [
                    ("fc1", torch.nn.Linear(16, 120)),
                    ("fc_relu_1", torch.nn.ReLU()),
                    ("fc2", torch.nn.Linear(120, 84)),
                    ("fc_relu_2", torch.nn.ReLU()),
                    ("fc3", torch.nn.Linear(84, 10)),
                    ("fc_log_softmax", torch.nn.LogSoftmax(dim=1)),
                ]
            )
        )

    def forward(self, x):
        """Forward pass of the model.

        Args:
            x ([type]): [description]

        Returns:
            [type]: [description]
        """
        x = self.backbone(x)
        x = self.head(x)
        return x

## Named Modules

Returns an iterator over all modules in the network, yielding both the name of the module as well as the module itself.

In [129]:
for name, layer in ToySequentialModel().named_modules():
    print(name)


backbone
backbone.cl1
backbone.cl_relu1
backbone.cl2
backbone.cl_relu2
head
head.fc1
head.fc_relu_1
head.fc2
head.fc_relu_2
head.fc3
head.fc_log_softmax


## Get Convolutional Layers

In [130]:
def get_conv_layers(
    model: Callable, layer_type: str = "Conv2d"
) -> Dict[str, str]:
    """Create a function that give me the convolutional layers of PyTorch model.

    This function is created to be used in conjunction with Visualization of Feature Maps.

    Args:
        model (Union[torchvision.models, timm.models]): A PyTorch model.
        layer_type (str): The type of layer to be extracted.

    Returns:
        conv_layers (Dict[str, str]): {"layer1.0.conv1": layer1.0.conv1, ...}

    Example:
        >>> resnet18_pretrained_true = timm.create_model(model_name = "resnet34", pretrained=True, num_classes=10).to(DEVICE)
        >>> conv_layers = get_conv_layers(resnet18_pretrained_true, layer_type="Conv2d")
    """

    if layer_type == "Conv2d":
        _layer_type = torch.nn.Conv2d
    elif layer_type == "Conv1d":
        _layer_type = torch.nn.Conv1d

    conv_layers = {}
    for name, layer in model.named_modules():
        if isinstance(layer, _layer_type):
            conv_layers[name] = name
    return conv_layers

In [131]:
>>> resnet18_pretrained_true = timm.create_model(model_name = "resnet34", pretrained=True, num_classes=10).to(DEVICE)
>>> conv_layers = get_conv_layers(resnet18_pretrained_true, layer_type="Conv2d")
>>> print(conv_layers)

{'conv1': 'conv1', 'layer1.0.conv1': 'layer1.0.conv1', 'layer1.0.conv2': 'layer1.0.conv2', 'layer1.1.conv1': 'layer1.1.conv1', 'layer1.1.conv2': 'layer1.1.conv2', 'layer1.2.conv1': 'layer1.2.conv1', 'layer1.2.conv2': 'layer1.2.conv2', 'layer2.0.conv1': 'layer2.0.conv1', 'layer2.0.conv2': 'layer2.0.conv2', 'layer2.0.downsample.0': 'layer2.0.downsample.0', 'layer2.1.conv1': 'layer2.1.conv1', 'layer2.1.conv2': 'layer2.1.conv2', 'layer2.2.conv1': 'layer2.2.conv1', 'layer2.2.conv2': 'layer2.2.conv2', 'layer2.3.conv1': 'layer2.3.conv1', 'layer2.3.conv2': 'layer2.3.conv2', 'layer3.0.conv1': 'layer3.0.conv1', 'layer3.0.conv2': 'layer3.0.conv2', 'layer3.0.downsample.0': 'layer3.0.downsample.0', 'layer3.1.conv1': 'layer3.1.conv1', 'layer3.1.conv2': 'layer3.1.conv2', 'layer3.2.conv1': 'layer3.2.conv1', 'layer3.2.conv2': 'layer3.2.conv2', 'layer3.3.conv1': 'layer3.3.conv1', 'layer3.3.conv2': 'layer3.3.conv2', 'layer3.4.conv1': 'layer3.4.conv1', 'layer3.4.conv2': 'layer3.4.conv2', 'layer3.5.conv1':

In [132]:
activation = {}

def get_intermediate_features(name: str) -> Callable:
    """Get the intermediate features of a model. Forward Hook.

    This is using forward hook with reference https://discuss.pytorch.org/t/how-can-l-load-my-best-model-as-a-feature-extractor-evaluator/17254/5

    Args:
        name (str): name of the layer.

    Returns:
        Callable: [description]
    """

    def hook(model, input, output):
        activation[name] = output.detach()

    return hook


# The below is testing the forward hook functionalities, especially getting intermediate features.
# Note that both models are same organically but created differently.
# Due to seeding issues, you can check whether they are the same output or not by running them separately.
# We also used assertion to check that the output from model(x) is same as torch.nn.LogSoftmax(dim=1)(fc3_output)

use_sequential_model = True
x = torch.randn(1, 25)

if not use_sequential_model:

    model = ToyModel()

    model.fc2.register_forward_hook(get_intermediate_features("fc2"))
    model.fc3.register_forward_hook(get_intermediate_features("fc3"))
    output = model(x)
    print(activation)
    fc2_output = activation["fc2"]
    fc3_output = activation["fc3"]
    # assert output and logsoftmax fc3_output are the same
    assert torch.allclose(output, torch.nn.LogSoftmax(dim=1)(fc3_output))
else:
    sequential_model = ToySequentialModel()

    # Do this if you want all, if not you can see below.
    # for name, layer in sequential_model.named_modules():
    #     layer.register_forward_hook(get_intermediate_features(name))
    sequential_model.head.fc2.register_forward_hook(
        get_intermediate_features("head.fc2")
    )
    sequential_model.head.fc3.register_forward_hook(
        get_intermediate_features("head.fc3")
    )
    sequential_model_output = sequential_model(x)
    print(activation)
    fc2_output = activation["head.fc2"]
    fc3_output = activation["head.fc3"]
    assert torch.allclose(
        sequential_model_output, torch.nn.LogSoftmax(dim=1)(fc3_output)
    )


{'head.fc2': tensor([[ 0.0697,  0.0544, -0.0157, -0.1059, -0.0464, -0.0090,  0.0532, -0.1273,
         -0.0286, -0.0151,  0.0963,  0.2205,  0.0745, -0.0110, -0.1127, -0.0367,
         -0.0681,  0.0463, -0.0833,  0.1288,  0.1058,  0.0976, -0.0251,  0.0980,
         -0.0110,  0.1170, -0.0650,  0.2091, -0.1773,  0.0363, -0.1452,  0.0036,
          0.0112, -0.0304, -0.0620, -0.0658, -0.0543,  0.0072,  0.0436,  0.0703,
          0.0254, -0.0614,  0.0164, -0.1003, -0.0396,  0.0349,  0.0089, -0.1243,
         -0.1037, -0.0491,  0.0627, -0.1347,  0.0010, -0.1290, -0.0280, -0.0344,
          0.1487, -0.1764, -0.0233,  0.0082,  0.1270,  0.0368,  0.0103, -0.0929,
          0.0038,  0.1346, -0.0688, -0.0437, -0.1205, -0.1596, -0.0240, -0.1001,
         -0.0300, -0.1119,  0.0344, -0.1587,  0.0329, -0.0424,  0.0999,  0.0732,
          0.1116,  0.0220, -0.0570,  0.0232]]), 'head.fc3': tensor([[ 0.0256, -0.0924,  0.0456,  0.0972,  0.0107,  0.0527,  0.0208,  0.0373,
          0.0451,  0.0712]])}
